In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pickle

In [3]:
data = pd.read_csv('Churn_Modelling.csv')

data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Drop PII (Irelavant Feature)

data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
# Encode Categorical Data

label_gender = LabelEncoder()
data['Gender'] = label_gender.fit_transform(data['Gender'])

# One hot Encode - Geography Column - if Geography column uses Label Encoder, then there are chances that Germany is assigned to 2, spain with 1.
# In an ANN model, this means data with higher value takes precedence which should not happen in our case

encoder_geography=OneHotEncoder()
encoded_values = encoder_geography.fit_transform(data[['Geography']])
encoded_values.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
encoder_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
encoded_df = pd.DataFrame(encoded_values.toarray(),columns=encoder_geography.get_feature_names_out(['Geography']))

In [8]:
# Combine one Hot Encoded data with the original data

data = pd.concat([data.drop(['Geography'],axis=1),encoded_df],axis=1)

data.head(10)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
5,645,1,44,8,113755.78,2,1,0,149756.71,1,0.0,0.0,1.0
6,822,1,50,7,0.00,2,1,1,10062.80,0,1.0,0.0,0.0
7,376,0,29,4,115046.74,4,1,0,119346.88,1,0.0,1.0,0.0
8,501,1,44,4,142051.07,2,0,1,74940.50,0,1.0,0.0,0.0
9,684,1,27,2,134603.88,1,1,1,71725.73,0,1.0,0.0,0.0


In [9]:
# Train - test Split Data with Dependent and Independent Features

x= data.drop(['EstimatedSalary'],axis=1)
y = data['EstimatedSalary']

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=42, test_size= 0.2)

# Scaling the Features

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [10]:
with open('scaler_regression_data.pkl','wb') as file:
    pickle.dump(scaler,file)

In [14]:
# ANN regression Problem Statement

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model =Sequential(
[
    Dense(x_train.shape[1],activation='relu',input_shape=(x_train.shape[1],)), # Input Layer

    Dense(62,activation='relu'), # First hidden layer Connected with input layer

    Dense(32,activation='relu'), # Second Hidden layer

    Dense(1) # Output layer -- No Activation Function ( Linear activation function by default) - which means the predicted value is recevied
]
)

c:\Users\Swetha\OneDrive\Desktop\Python & ML Related\Deep Learning\DL - Udemy\venv\lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Compile the model

model.compile(
    optimizer='adam',
    loss = 'mean_absolute_error',
    metrics =['mae']
)

In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 12)             │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 62)             │           806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,011 (11.76 KB)

 Trainable params: 3,011 (11.76 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Set the logs

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir ='regressionlogs/fit/'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensor_call_back = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [19]:
# Set Up Early Stopping

early_stopping = EarlyStopping(
    monitor ='val_loss',
    patience = 5,
    restore_best_weights = True
)

In [20]:

history = model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    callbacks=[tensor_call_back,early_stopping]
    )

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 99989.1562 - mae: 99989.1562 - val_loss: 96422.5078 - val_mae: 96422.5078
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 87557.6328 - mae: 87557.6328 - val_loss: 69607.7734 - val_mae: 69607.7734
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 59074.1758 - mae: 59074.1758 - val_loss: 51331.3320 - val_mae: 51331.3320
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 51800.7266 - mae: 51800.7266 - val_loss: 50333.1680 - val_mae: 50333.1680
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 50894.2383 - mae: 50894.2383 - val_loss: 50216.3516 - val_mae: 50216.3516
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 50548.0547 - mae: 50548.0547 - val_loss: 50171.4023 - val_mae: 50171.4023
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 50361.1797 - mae: 50361.1797 - val_loss: 50181.4297 - val_mae: 50181.4297
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - 

In [41]:
%reload_ext tensorboard

In [43]:
%tensorboard --logdir regressionlogs/fit

ERROR: Failed to launch TensorBoard (exited with 1).

In [44]:
model.save('regression_model.h5')